In [1]:
# @title Install libraries
!pip install sentence_transformers transformers langchain einops
!pip install accelerate bitsandbytes trl peft loralib
!pip install chromadb faiss-gpu pacmap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 974.6/974.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.8/315.8 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.3/125.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 2.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# @title Load data

from io import BytesIO
from IPython.display import display

import requests
import pandas as pd

gg_id = '814552626'
file_id = '11Xrvp1vdB0UJ34CIM_A-hKAnCJp_0BQA0HF6GGTerRc'
file_path = f'https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv&gid={gg_id}'

df = pd.read_csv(file_path)
df = df.ffill()
df['Id'] = df.reset_index(drop=False)['index'].astype(str).str.zfill(5)
df

,Object,Detail,Interpretation,Id
0,Ảnh,Đốt ảnh,Tình duyên sắp thành,00000
1,Ảnh,Tặng ảnh,Điềm chia ly / sắp đi xa,00001
2,Ảnh,Được tặng ảnh,Điềm chia ly / sắp đi xa,00002
3,Ảnh,Ảnh người yêu,Có tình địch,00003
4,Ảnh,Xé ảnh người khác,Sắp hội ngộ người đó,00004
...,...,...,...,...
2303,Xưởng,Trông nom xưởng thở,Tài lộc hanh thông,02303
2304,Xưởng,Đi ra khỏi xưởng,Bạn sẽ được quý nhân cất nhắc,02304
2305,Xưởng,Đi ra khỏi xưởng rồi quay lại,Bạn đang được chủ tin cậy,02305
2306,Yêu,Yêu tha thiết 1 người mà người ấy luôn ruồng rẫy,Hôn nhân sắp thành,02306


In [4]:
# @title Load language model with Quantization
import torch
from transformers import pipeline
from transformers import AutoConfig, BitsAndBytesConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from sentence_transformers import SentenceTransformer

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_use_double_quant=True,
#     bnb_4bit_quant_type="nf4",
#     bnb_4bit_compute_dtype=torch.bfloat16,
# )

# model_name = "vinai/PhoGPT-7B5-Instruct"
model_name = "vinai/PhoGPT-4B-Chat"
config = AutoConfig.from_pretrained(model_name, trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True, config=config, load_in_8bit=True)
model

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.19k [00:00<?, ?B/s]

configuration_mpt.py:   0%|          | 0.00/16.4k [00:00<?, ?B/s]

attention.py:   0%|          | 0.00/24.6k [00:00<?, ?B/s]

flash_attn_triton.py:   0%|          | 0.00/28.2k [00:00<?, ?B/s]

...

custom_embedding.py:   0%|          | 0.00/292 [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- custom_embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/vinai/PhoGPT-4B-Chat:
- modeling_mpt.py
- adapt_tokenizer.py
- hf_prefixlm_converter.py
- meta_init_context.py
- param_init_fns.py
- custom_embedding.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


pytorch_model.bin:   0%|          | 0.00/7.38G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/91.0 [00:00<?, ?B/s]

MPTForCausalLM(
  (transformer): MPTModel(
    (wte): SharedEmbedding(20480, 3072)
    (emb_drop): Dropout(p=0.0, inplace=False)
    (blocks): ModuleList(
      (0-31): 32 x MPTBlock(
        (norm_1): LPLayerNorm((3072,), eps=1e-05, elementwise_affine=True)
        (attn): MultiheadAttention(
          (Wqkv): Linear8bitLt(in_features=3072, out_features=9216, bias=True)
          (out_proj): Linear8bitLt(in_features=3072, out_features=3072, bias=True)
        )
        (norm_2): LPLayerNorm((3072,), eps=1e-05, elementwise_affine=True)
        (ffn): MPTMLP(
          (up_proj): Linear8bitLt(in_features=3072, out_features=12288, bias=True)
          (down_proj): Linear8bitLt(in_features=12288, out_features=3072, bias=True)
        )
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_ffn_dropout): Dropout(p=0.0, inplace=False)
      )
    )
    (norm_f): LPLayerNorm((3072,), eps=1e-05, elementwise_affine=True)
  )
)

In [5]:
# @title Dataset
# Reference: https://github.com/VinAIResearch/PhoGPT
from tqdm import tqdm

samples = []
for symbol, meaning in tqdm(df[['Detail', 'Interpretation']].values):
    sample = [
        {"role": "user", "content": f"Ý nghĩa khi mơ thấy {symbol.lower()}",},
        {"role": "assistant", "content": meaning,},
    ]
    sample = tokenizer.apply_chat_template(sample, tokenize=False, add_generation_prompt=True)
    samples.append(sample)

samples_df = pd.DataFrame(list(zip(samples)), columns=['text'])
display(samples_df)

from datasets import load_dataset, Dataset
dataset = Dataset.from_pandas(samples_df)
dataset

100%|██████████| 2308/2308 [00:00<00:00, 8703.96it/s]


,text
0,### Câu hỏi: Ý nghĩa khi mơ thấy đốt ảnh\n### ...
1,### Câu hỏi: Ý nghĩa khi mơ thấy tặng ảnh\n###...
2,### Câu hỏi: Ý nghĩa khi mơ thấy được tặng ảnh...
3,### Câu hỏi: Ý nghĩa khi mơ thấy ảnh người yêu...
4,### Câu hỏi: Ý nghĩa khi mơ thấy xé ảnh người ...
...,...
2303,### Câu hỏi: Ý nghĩa khi mơ thấy trông nom xưở...
2304,### Câu hỏi: Ý nghĩa khi mơ thấy đi ra khỏi xư...
2305,### Câu hỏi: Ý nghĩa khi mơ thấy đi ra khỏi xư...
2306,### Câu hỏi: Ý nghĩa khi mơ thấy yêu tha thiết...


Dataset({
    features: ['text'],
    num_rows: 2308
})

In [6]:
# @title LoRA Configuration
from peft import LoraConfig, PeftModel, AutoPeftModelForCausalLM

lora_config = LoraConfig(
    bias="none",
    r=32,
    lora_alpha=32,
    lora_dropout=0.05,
    # init_lora_weights='pissa',
    use_rslora=True,
    target_modules=["attn.Wqkv", "attn.out_proj", "ffn.up_proj", "ffn.down_proj"],
    task_type="CAUSAL_LM"
)

In [7]:
# @title Training Configuration
from transformers import TrainingArguments
from trl import SFTTrainer

output_model = model_name.split('/')[-1] + "-dreamerpretation-v1"
# output_model

training_arguments = TrainingArguments(
    output_dir=output_model,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=4,
    learning_rate=69e-6,
    # warmup_ratio=0.03,
    # max_grad_norm=0.35,
    # group_by_length=True,
    optim="paged_adamw_32bit",
    lr_scheduler_type="cosine",
    # lr_scheduler_kwargs=dict(),
    logging_strategy="epoch",
    logging_steps=10,
    save_strategy="epoch",
    save_steps=10,
    save_only_model=True,
    save_total_limit=10,
    num_train_epochs=25,
    max_steps=-1,
    fp16=True,
    # resume_from_checkpoint='',
    # load_best_model_at_end=True,
    # report_to="tensorboard",
    # push_to_hub=True,
)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    args=training_arguments,
    peft_config=lora_config,
    train_dataset=dataset,
    dataset_text_field=list(dataset.features.keys())[0],
    packing=False,
    max_seq_length=256,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1965: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the

Map:   0%|          | 0/2308 [00:00<?, ? examples/s]

In [8]:
# @title Training
%%time

trainer.train()

/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/116013fa63f8c4025739487e1cbff65b7375bbe2/configuration_mpt.py:141: UserWarning: If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".
  warnings.warn(UserWarning('If not using a Prefix Language Model, we recommend setting "attn_impl" to "flash" instead of "triton".'))
/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/116013fa63f8c4025739487e1cbff65b7375bbe2/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagating key_padding_mask to the attention module ' + 'and applying it within the attention module can cause ' + 'unnecessary computation/memory usage. Consider integrating ' + 'into attn_bias once and pa

Step,Training Loss
72,1.789000
144,1.267100
216,1.141200
289,0.971300
361,0.820700
433,0.691500
505,0.600600
578,0.525700
650,0.485500
722,0.448000


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/116013fa63f8c4025739487e1cbff65b7375bbe2/configuration_mpt.py:114: UserWarning: alibi or rope is turned on, setting `learned_pos_emb` to `False.`
  warnings.warn(f'alibi or rope is turned on, setting `learned_pos_emb` to `False.`')
/root/.cache/huggingface/modules/transformers_modules/vinai/PhoGPT-4B-Chat/116013fa63f8c4025739487e1cbff65b7375bbe2/attention.py:87: UserWarning: Propagating key_padding_mask to the attention module and applying it within the attention module can cause unnecessary computation/memory usage. Consider integrating into attn_bias once and passing that to each attention module instead.
  warnings.warn('Propagati

CPU times: user 1h 17min 50s, sys: 20min 58s, total: 1h 38min 48s
Wall time: 1h 41min 12s


TrainOutput(global_step=1800, training_loss=0.5725207413567437, metrics={'train_runtime': 6070.7175, 'train_samples_per_second': 9.505, 'train_steps_per_second': 0.297, 'total_flos': 4.78770782662656e+16, 'train_loss': 0.5725207413567437, 'epoch': 24.913494809688583})

In [10]:
# @title Checkpoint Packaging
import os
import sys
import zipfile


def compress_folder(target_dir, folder_name = None):

    if folder_name is None:
        folder_name = target_dir.split('/')[-1]

    zip_obj = zipfile.ZipFile(folder_name + '.zip', 'w', zipfile.ZIP_DEFLATED)
    root_len = len(target_dir) + 1
    for base, dirs, files in os.walk(target_dir):
        for file in files:
            fn = os.path.join(base, file)
            zip_obj.write(fn, fn[root_len:])


target_dir = "/content/PhoGPT-4B-Chat-dreamerpretation-v1/checkpoint-1734"
folder_name = "PhoGPT-4B-Chat-dreamerpretation-v1"

compress_folder(target_dir, folder_name)